# Recurrent Neural Network

## Part 1 Data Preprocessing

In [1]:
# Importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Importing the training sets
training_data = pd.read_csv("datas/Google_Stock_train.csv")
training_data.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [3]:
training_set = training_data.iloc[:,1:2]
training_set.head()

,Open
0,325.25
1,331.27
2,329.83
3,328.34
4,322.04


#### Feature Scalling

In [4]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_data_scaled = sc.fit_transform(training_set)

In [5]:
print(training_data.shape)

(1258, 6)


In [6]:
## Creating a data structure with 60 timestemps and 1 output
x_train = []
y_train = []
for i in range(60, 1258):
    # 1258 is shape of training data
    x_train.append (training_data_scaled[i-60:i,0])
    y_train.append(training_data_scaled[i,0])
# convert to numpy 
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape 
# 
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))

In [7]:
x_train[0,:5], y_train[0], len(x_train[0])

(array([[0.08581368],
        [0.09701243],
        [0.09433366],
        [0.09156187],
        [0.07984225]]),
 0.08627874097775134,
 60)

# Part 2 Building the RNN

In [8]:
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [9]:
# Initialising the RNN 
# Sequential API
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units =50, return_sequences = True, input_shape = (x_train.shape[1],1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Drop regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularization
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compilling the RNN
regressor.compile(optimizer = "adam", loss = "mean_squared_error")

# Fitting the RNN to the Trainig set
regressor.fit(x_train, y_train, epochs= 100, batch_size = 32)


Epoch 1/100
38/38 [==============================] - 7s 39ms/step - loss: 0.1111
Epoch 2/100
38/38 [==============================] - 1s 38ms/step - loss: 0.0073
Epoch 3/100
38/38 [==============================] - 1s 38ms/step - loss: 0.0058
Epoch 4/100
38/38 [==============================] - 1s 38ms/step - loss: 0.0053
Epoch 5/100
38/38 [==============================] - 1s 38ms/step - loss: 0.0055
Epoch 6/100
38/38 [==============================] - 1s 38ms/step - loss: 0.0048
Epoch 7/100
38/38 [==============================] - 1s 38ms/step - loss: 0.0048
Epoch 8/100
38/38 [==============================] - 1s 38ms/step - loss: 0.0043
Epoch 9/100
38/38 [==============================] - 1s 39ms/step - loss: 0.0055
Epoch 10/100
38/38 [==============================] - 2s 41ms/step - loss: 0.0045
Epoch 11/100
38/38 [==============================] - 2s 41ms/step - loss: 0.0038
Epoch 12/100
38/38 [==============================] - 1s 39ms/step - loss: 0.0049
Epoch 13/100
38/38 [=====

38/38 [==============================] - 2s 46ms/step - loss: 0.0016


## Making the prediction

In [10]:
# Loadiing the real stock price of 2017
test_data = pd.read_csv("datas/Google_Stock_test.csv")
test_data.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"


In [19]:
# use only Open data
dataset_test = test_data.iloc[:, 1:2].values

In [20]:
# Getting the predicted stock price of 2017
dataset_total = pd.concat((training_data["Open"], test_data["Open"]),axis = 0)

In [21]:
dataset_total.head()

0    325.25
1    331.27
2    329.83
3    328.34
4    322.04
Name: Open, dtype: float64

In [23]:
inputs = dataset_total[len(dataset_total) - len(dataset_test) -60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
x_test = []
for i in range(60,80):
    x_test.append(inputs[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
predicted_stock_price = regressor.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

## Visualising teh Results

In [ ]:
plt